In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

# 設定圖表的美觀樣式
sns.set_theme(style="whitegrid")

# --- 載入資料 ---
file_path = '2024_fb_ads_president_scored_anon.csv'
df = pd.read_csv(file_path, low_memory=False)

# --- 資料清理與轉型 (非常重要的一步) ---
# 將 'spend' 欄位轉為數值，忽略無法轉換的錯誤
df['spend'] = pd.to_numeric(df['spend'], errors='coerce')

# 將 'publisher_platforms' 從字串轉為列表
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return None # 如果解析失敗，回傳 None

df['platforms_list'] = df['publisher_platforms'].apply(safe_literal_eval)

print("資料載入與初步清理完成！")
df[['page_name', 'spend', 'platforms_list']].head()

KeyError: 'spend'

In [ ]:
# 為了讓圖表更清晰，我們先過濾掉一些極端的離群值
# 這裡我們只看花費小於 2000 的廣告
df_filtered = df[df['spend'] < 2000]

# --- 繪製直方圖 ---
plt.figure(figsize=(12, 6)) # 設定圖表大小
sns.histplot(df_filtered['spend'], bins=50, kde=True)
plt.title('Distribution of Ad Spend (spend < 2000)') # 設定標題
plt.xlabel('Spend') # 設定 X 軸標籤
plt.ylabel('Number of Ads') # 設定 Y 軸標籤
plt.show() # 顯示圖表

# --- 繪製箱型圖 ---
plt.figure(figsize=(12, 6))
sns.boxplot(x=df_filtered['spend'])
plt.title('Boxplot of Ad Spend (spend < 2000)')
plt.xlabel('Spend')
plt.show()

In [1]:
# .explode() 會將列表中的每個平台拆分成獨立的行
# .value_counts() 會計算每個平台出現的次數
platform_counts = df.explode('platforms_list')['platforms_list'].value_counts()

# --- 繪製條形圖 ---
plt.figure(figsize=(10, 6))
sns.barplot(x=platform_counts.index, y=platform_counts.values)
plt.title('Most Popular Publisher Platforms')
plt.xlabel('Platform')
plt.ylabel('Number of Ad Placements')
plt.xticks(rotation=45) # 讓 X 軸的標籤旋轉，避免重疊
plt.show()

NameError: name 'df' is not defined

In [ ]:
# --- 找出花費最高的前 10 個粉絲專頁 ---
top_spenders = df.groupby('page_name')['spend'].sum().nlargest(10).index
df_top = df[df['page_name'].isin(top_spenders)]

# --- 計算每個專頁在各平台的投放數 ---
# .crosstab() 是製作交叉分析表的利器
platform_crosstab = pd.crosstab(
    df_top.explode('platforms_list')['page_name'],
    df_top.explode('platforms_list')['platforms_list']
)

# --- 繪製堆疊條形圖 ---
platform_crosstab.plot(kind='bar', stacked=True, figsize=(15, 8))
plt.title('Publisher Platform Usage for Top 10 Spenders')
plt.xlabel('Page Name')
plt.ylabel('Number of Ad Placements')
plt.xticks(rotation=45, ha='right') # ha='right' 讓標籤對齊
plt.legend(title='Platform')
plt.tight_layout() # 自動調整邊距
plt.show()